In [1]:
import pandas as pd
import seaborn as sns

train = pd.read_csv("/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/train.csv")
test = pd.read_csv("/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/test.csv")

log_train = pd.read_csv("/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/log_train.csv")
log_test = pd.read_csv("/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/log_test.csv")

In [2]:
log_train_df= log_train
train_df = train

In [3]:
log_train_df['time'] = pd.to_datetime(log_train_df['time'])

# Extract features
log_train_df['hour'] = log_train_df['time'].dt.hour
log_train_df['day_of_week'] = log_train_df['time'].dt.dayofweek
log_train_df['day'] = log_train_df['time'].dt.day

# Count of events per ID
event_count = log_train_df.groupby('ID')['event'].count().reset_index(name='event_count')

# Count of unique sources per ID
unique_sources = log_train_df.groupby('ID')['source'].nunique().reset_index(name='unique_sources')

# Merge these features back to the original dataset on ID
train_df_enriched = train_df.merge(event_count, on='ID', how='left')
train_df_enriched = train_df_enriched.merge(unique_sources, on='ID', how='left')


# Time difference features (e.g., average time between events for each ID)
log_train_df['time_diff'] = log_train_df.sort_values(['ID', 'time']).groupby('ID')['time'].diff().dt.total_seconds()
avg_time_diff = log_train_df.groupby('ID')['time_diff'].mean().reset_index(name='avg_time_diff')

# Fill missing values for IDs that did not have event data


train_df_enriched = train_df_enriched.merge(avg_time_diff, on='ID', how='left')

# Fill missing values for IDs that did not have event data
train_df_enriched.fillna({'event_count': 0, 'unique_sources': 0, "avg_time_diff":0}, inplace=True)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/reshape/merge.py:1203: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():


In [4]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import ADASYN

# Assuming 'train_df_enriched' is your dataset
train_clean = train_df_enriched.dropna(subset=['label'])

X = train_clean.drop(['label', 'ID'], axis=1)
y = train_clean['label']

# Initialize the SimpleImputer with 'median' strategy
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)
X_imputed_df = pd.DataFrame(X_imputed, columns=X.columns)

# Resampling
X_resampled, y_resampled = ADASYN().fit_resample(X_imputed_df, y)

In [7]:
import optuna
import lightgbm as lgb
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

# Assuming X_resampled and y_resampled are your features and target after resampling

def objective(trial):
    # Splitting data for hyperparameter optimization
    X_train, X_valid, y_train, y_valid = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

    # Defining hyperparameter search space
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': -1,
        'boosting_type': 'dart',
        'learning_rate': 0.01,
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': -1,
        'min_child_samples': 30,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'reg_alpha': 0.1,
        'reg_lambda': 0.1,
    }

    # Training the model with the trial's parameters
    gbm = lgb.train(param, lgb.Dataset(X_train, label=y_train),num_boost_round=2000, valid_sets=[lgb.Dataset(X_valid, label=y_valid)],callbacks=[lgb.early_stopping(stopping_rounds=100),lgb.log_evaluation(period=100)])
    preds = gbm.predict(X_valid)
    # Calculating log loss for the validation set
    loss = log_loss(y_valid, preds)
    return loss

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100) # You can adjust the number of trials

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)


[I 2024-03-29 14:44:23,237] A new study created in memory with name: no-name-bb9086e7-bed0-48d2-a2ef-6f40763da230


[100]	valid_0's binary_logloss: 0.509639
[200]	valid_0's binary_logloss: 0.444115
[300]	valid_0's binary_logloss: 0.386128
[400]	valid_0's binary_logloss: 0.342502
[500]	valid_0's binary_logloss: 0.30764
[600]	valid_0's binary_logloss: 0.295469
[700]	valid_0's binary_logloss: 0.27751
[800]	valid_0's binary_logloss: 0.267083
[900]	valid_0's binary_logloss: 0.25462
[1000]	valid_0's binary_logloss: 0.247709
[1100]	valid_0's binary_logloss: 0.240592
[1200]	valid_0's binary_logloss: 0.237612
[1300]	valid_0's binary_logloss: 0.233295
[1400]	valid_0's binary_logloss: 0.227856
[1500]	valid_0's binary_logloss: 0.226343
[1600]	valid_0's binary_logloss: 0.222975
[1700]	valid_0's binary_logloss: 0.220052
[1800]	valid_0's binary_logloss: 0.218031
[1900]	valid_0's binary_logloss: 0.216299
[2000]	valid_0's binary_logloss: 0.215166


[I 2024-03-29 14:56:41,381] Trial 0 finished with value: 0.21516608769567755 and parameters: {'num_leaves': 49}. Best is trial 0 with value: 0.21516608769567755.
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lightgbm/callback.py:325: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


[100]	valid_0's binary_logloss: 0.492814
[200]	valid_0's binary_logloss: 0.422461
[300]	valid_0's binary_logloss: 0.362375
[400]	valid_0's binary_logloss: 0.320208
[500]	valid_0's binary_logloss: 0.286437
[600]	valid_0's binary_logloss: 0.276333
[700]	valid_0's binary_logloss: 0.25983
[800]	valid_0's binary_logloss: 0.250946
[900]	valid_0's binary_logloss: 0.240397
[1000]	valid_0's binary_logloss: 0.234968
[1100]	valid_0's binary_logloss: 0.229205
[1200]	valid_0's binary_logloss: 0.226857
[1300]	valid_0's binary_logloss: 0.223622
[1400]	valid_0's binary_logloss: 0.219382
[1500]	valid_0's binary_logloss: 0.218377
[1600]	valid_0's binary_logloss: 0.215594
[1700]	valid_0's binary_logloss: 0.213287
[1800]	valid_0's binary_logloss: 0.211861
[1900]	valid_0's binary_logloss: 0.210516
[2000]	valid_0's binary_logloss: 0.209722


[I 2024-03-29 15:18:24,573] Trial 1 finished with value: 0.20972225092827104 and parameters: {'num_leaves': 188}. Best is trial 1 with value: 0.20972225092827104.
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lightgbm/callback.py:325: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


...

[100]	valid_0's binary_logloss: 0.490921
[200]	valid_0's binary_logloss: 0.420176
[300]	valid_0's binary_logloss: 0.359755
[400]	valid_0's binary_logloss: 0.317366
[500]	valid_0's binary_logloss: 0.28389
[600]	valid_0's binary_logloss: 0.27371
[700]	valid_0's binary_logloss: 0.257088
[800]	valid_0's binary_logloss: 0.248404
[900]	valid_0's binary_logloss: 0.237975
[1000]	valid_0's binary_logloss: 0.232477
[1100]	valid_0's binary_logloss: 0.22698
[1200]	valid_0's binary_logloss: 0.224751
[1300]	valid_0's binary_logloss: 0.221617
[1400]	valid_0's binary_logloss: 0.217393
[1500]	valid_0's binary_logloss: 0.216413
[1600]	valid_0's binary_logloss: 0.213732
[1700]	valid_0's binary_logloss: 0.21155
[1800]	valid_0's binary_logloss: 0.210055
[1900]	valid_0's binary_logloss: 0.20879
[2000]	valid_0's binary_logloss: 0.208031


[I 2024-03-29 21:11:34,072] Trial 18 finished with value: 0.20803093409043288 and parameters: {'num_leaves': 267}. Best is trial 11 with value: 0.20738547176662459.
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lightgbm/callback.py:325: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


[100]	valid_0's binary_logloss: 0.494433
[200]	valid_0's binary_logloss: 0.424547
[300]	valid_0's binary_logloss: 0.36488
[400]	valid_0's binary_logloss: 0.32236
[500]	valid_0's binary_logloss: 0.288437
[600]	valid_0's binary_logloss: 0.27807
[700]	valid_0's binary_logloss: 0.261526


[W 2024-03-29 21:16:45,350] Trial 19 failed with parameters: {'num_leaves': 150} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/w5/7276v8b14pj5_4lrc2r9xkvr0000gn/T/ipykernel_26500/2684284992.py", line 29, in objective
    gbm = lgb.train(param, lgb.Dataset(X_train, label=y_train),num_boost_round=2000, valid_sets=[lgb.Dataset(X_valid, label=y_valid)],callbacks=[lgb.early_stopping(stopping_rounds=100),lgb.log_evaluation(period=100)])
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lightgbm/engine.py", line 276, in train
    booster.update(fobj=fobj)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lightgbm/basic.py", line 3891, in update
    _safe_call(_LIB.LGBM_BoosterUpdateOne

KeyboardInterrupt: 

In [ ]:
#'num_leaves': 300